# Python for Learning AI Week 2: TCP Server

Welcome to Week 2's exploration of Port Communication in Python! This notebook focuses on server-side programming, teaching you how to build and run TCP and HTTP servers. Understanding how servers work is essential for building networked applications, APIs, and AI-powered services.

## What You'll Learn
1. Understanding ports and network communication
2. Creating TCP servers that listen on ports
3. Handling client connections
4. Building a simple HTTP server
5. Responding to different types of requests

## Prerequisites
- Basic Python knowledge ([covered in Week 1](../week1/week1_python_basics.ipynb))
- Understanding of functions and error handling

Let's start by installing the required packages:

In [ ]:
# Check if required packages are installed
try:
    import socket  # For creating network connections
    import time    # For time-related functions
    import psutil  # For system and process utilities
    from http.server import HTTPServer, BaseHTTPRequestHandler  # For HTTP server
    import json    # For formatting HTTP responses
    print("✓ All required packages are available!")
except ImportError as e:
    print(f"✗ Missing package: {str(e)}")
    print("\nIMPORTANT: To install packages, run this command in your terminal (not in the notebook):")
    print("pip install psutil")
    print("\nWhy not install in the notebook?")
    print("- Installing via notebook (!pip install) only affects the current kernel")
    print("- Installing in the terminal ensures packages are available everywhere")
    print("- This ensures consistency across all notebooks and scripts")
    print("\nAfter installing, restart the kernel to use the new package.")

## Understanding Ports and Network Communication

### Client-Server Communication Diagram

```
                        INTERNET/NETWORK
                              |
                              |
                              v
        +------------------+      Request      +------------------+
        |                  |  -------------->  |                  |
        |     CLIENT       |                   |      SERVER      |
        |                  |  <--------------  |                  |
        +------------------+      Response     +------------------+
               |                                   ^      ^
               |                                   |      |
               v                                   |      |
         Connects to                           Listening on
         specific port                         multiple ports
               |                                   |      |
               v                                   v      v
         +-------------+                     +-------+-------+
         |             |                     |       |       |
         | 192.168.1.5 |                     | :80   | :443  |
         |     :12345  |                     | HTTP  | HTTPS |
         +-------------+                     +-------+-------+
             Client                               Server
            IP:Port                              Ports
```

### Why Port Communication Matters

Port communication is foundational to modern computing for several key reasons:

1. **Enables Multi-Service Architecture**: Ports allow a single device to run multiple network services simultaneously
2. **Service Identification**: Helps identify which application or service should receive incoming data
3. **Security Boundary**: Provides a controlled interface for external systems to access specific services
4. **Network Troubleshooting**: Essential for diagnosing connectivity issues and network problems
5. **Application Development**: Critical knowledge for building distributed applications

### What is a Port?

A port is like a numbered doorway into your computer:
- Each port number (0-65535) can be used for different services
- Common ports: 80 (HTTP), 443 (HTTPS), 22 (SSH), 3306 (MySQL)
- Multiple applications can listen on different ports
- Only one application can use a specific port at a time

### Common Port Ranges:
1. Well-known ports (0-1023)
   - Reserved for standard services
   - Require administrative privileges
   - Examples: HTTP (80), HTTPS (443), FTP (21)

2. Registered ports (1024-49151)
   - Used by applications
   - Don't require special privileges
   - Good for custom applications

3. Dynamic ports (49152-65535)
   - Used for temporary connections
   - Available for any purpose

Let's explore what ports are currently in use on your system:

In [ ]:
def list_active_connections():
    """List all active network connections and their ports"""
    connections = psutil.net_connections()
    
    print("Active Network Connections:")
    print("-" * 50)
    print(f"{'Local Address':<20} {'Local Port':<10} {'Status':<10}")
    print("-" * 50)
    
    for conn in connections:
        if conn.laddr:  # Only show connections with local addresses
            addr = conn.laddr.ip
            port = conn.laddr.port
            status = conn.status
            print(f"{addr:<20} {port:<10} {status:<10}")

try:
    list_active_connections()
except Exception as e:
    print(f"Error listing connections: {e}")
    print("Note: Some systems may require elevated privileges to list all connections.")

## Creating a TCP Server

Let's create a basic TCP server that:
1. Listens on a specific port
2. Accepts connections from clients
3. Receives and sends messages
4. Properly closes connections

This example will help you understand:
- How servers listen for connections using ports
- How servers handle incoming client connections
- Basic socket programming concepts
- The server side of client-server communication

In [ ]:
def run_tcp_server(host='localhost', port=12345):
    """Run a TCP server that echoes back messages from clients"""
    # Create a socket object
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Allow port reuse (helpful for development)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    try:
        # Bind the socket to a specific address and port
        server_socket.bind((host, port))
        
        # Enable the server to accept connections
        # The parameter specifies how many unaccepted connections are allowed
        # before refusing new connections
        server_socket.listen(5)
        
        print(f"Server is listening on {host}:{port}")
        print("Waiting for client connections...")
        print("Press Ctrl+C to stop the server")
        
        # Continue accepting connections until interrupted
        while True:
            # Wait for a client to connect
            client_socket, address = server_socket.accept()
            print(f"Connection established with {address}")
            
            try:
                # Receive data from the client
                data = client_socket.recv(1024).decode('utf-8')
                if not data:
                    print("No data received")
                    continue
                    
                print(f"Received from {address}: {data}")
                
                # Send a response back to the client
                response = f"Server received: {data}"
                client_socket.send(response.encode('utf-8'))
                
            except Exception as e:
                print(f"Error handling client {address}: {e}")
            
            finally:
                # Close the connection
                client_socket.close()
                print(f"Connection with {address} closed")
            
    except KeyboardInterrupt:
        print("\nServer shutdown requested...")
    
    except Exception as e:
        print(f"Server error: {e}")
    
    finally:
        # Always close the server socket
        server_socket.close()
        print("Server stopped")

# Note: Run this function when you're ready to start the server
# It will continue running until you interrupt it (Ctrl+C or stop the kernel)

### Running the TCP Server

To start the TCP server, run the following cell. The server will:
1. Create a socket and bind it to the specified port
2. Listen for incoming connections
3. Accept and process client messages
4. Continue running until interrupted

Once the server is running, you can connect to it using the TCP client notebook.

In [ ]:
# Uncomment and run this cell when you're ready to start the server
# run_tcp_server()

## Creating a Simple HTTP Server

Now let's create a more advanced type of server: an HTTP server. HTTP servers are the foundation of web applications and APIs.

Creating an HTTP server will help you understand:
- How web servers process requests
- The HTTP request/response cycle
- Different HTTP methods (GET, POST)
- How web APIs handle data

Let's define our HTTP request handler:

In [ ]:
class SimpleHTTPHandler(BaseHTTPRequestHandler):
    def _send_response(self, message, status=200):
        """Helper method to send a response"""
        # Send HTTP status code (200 OK, 404 Not Found, etc.)
        self.send_response(status)
        
        # Set response headers
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        
        # Convert message to JSON and encode as bytes
        response = json.dumps({"message": message}).encode('utf-8')
        
        # Write response body to output stream
        self.wfile.write(response)
    
    def do_GET(self):
        """Handle GET requests"""
        # Route based on the requested path
        if self.path == '/':
            # Root path - welcome message
            self._send_response("Welcome to the simple HTTP server!")
            
        elif self.path == '/time':
            # Current time endpoint
            current_time = time.strftime("%Y-%m-%d %H:%M:%S")
            self._send_response(f"Current time is: {current_time}")
            
        else:
            # Any other path - 404 Not Found
            self._send_response("Not found", 404)
    
    def do_POST(self):
        """Handle POST requests"""
        if self.path == '/echo':
            # Get the length of the incoming data
            content_length = int(self.headers['Content-Length'])
            
            # Read the posted data
            post_data = self.rfile.read(content_length)
            
            # Echo it back in the response
            self._send_response(f"Received: {post_data.decode('utf-8')}")
            
        else:
            # Any other path - 404 Not Found
            self._send_response("Not found", 404)

### HTTP Request Handlers

The `SimpleHTTPHandler` class includes two important methods:

1. **`do_GET`**: Called when a client makes an HTTP GET request
   - Handles the root path `/` with a welcome message
   - Handles the `/time` path to return the current server time
   - Returns 404 for any other paths

2. **`do_POST`**: Called when a client makes an HTTP POST request
   - Handles the `/echo` path to echo back posted data
   - Returns 404 for any other paths

These methods are automatically called by the HTTP server framework based on the incoming request method.

In [ ]:
def run_http_server(port=8000):
    """Run an HTTP server"""
    # Create server with empty host ('') to listen on all interfaces
    server_address = ('', port)
    httpd = HTTPServer(server_address, SimpleHTTPHandler)
    
    print(f"Starting HTTP server on port {port}...")
    print("Press Ctrl+C to stop the server")
    
    try:
        # This will run until interrupted
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nShutting down server...")
    finally:
        httpd.server_close()
        print("Server closed")

### Running the HTTP Server

Now we can run our HTTP server. Once running, you can:
1. Access it from a web browser at http://localhost:8000/
2. Try different endpoints like http://localhost:8000/time
3. Test POST requests using the client notebook or tools like curl

In [ ]:
# Uncomment and run this cell when you're ready to start the HTTP server
# run_http_server()

## Testing Your Servers

Once your server is running, you can test it using:

1. The companion client notebook (`002_2_tcp_client.ipynb`)
2. A web browser (for HTTP server)
3. Command line tools like `curl` or `wget`
4. Any HTTP client library in Python

### Example curl commands for the HTTP server:
```bash
# Test GET request
curl http://localhost:8000/

# Test POST request
curl -X POST -d "Hello from curl!" http://localhost:8000/echo
```

## Conclusion: Understanding Servers in Network Communication

In this notebook, we've covered:

1. **What servers do**: Listen for and accept client connections
2. **How servers work**: Bind to ports and process incoming requests
3. **TCP servers**: Handle raw socket connections and data exchange
4. **HTTP servers**: Process web requests with standardized methods

Understanding server-side programming is essential for AI development as many AI systems are deployed as services that other applications can connect to. Whether you're creating an API for machine learning predictions or deploying a large language model as a service, the concepts covered here provide the foundation for that communication.

In the companion client notebook, you'll learn how to connect to these servers and exchange data with them.